In [7]:
from python_workflow_definition.aiida import write_workflow_json
from python_workflow_definition.jobflow import load_workflow_json
from aiida_workgraph import task, WorkGraph
from jobflow.managers.local import run_locally

from aiida import load_profile
load_profile()

workflow_json_filename =  "workflow_simple_aiida.json"

In [8]:
from simple_workflow import (
    add_x_and_y as _add_x_and_y, 
    add_x_and_y_and_z as _add_x_and_y_and_z,
)

In [9]:
def pickle_node(value):
    """Handle data nodes"""
    return value

helper_1 = task.pythonjob()(pickle_node)
helper_2 = task.pythonjob()(pickle_node)

add_x_and_y = task.pythonjob(outputs=["x", "y", "z"])(_add_x_and_y)
add_x_and_y_and_z = task.pythonjob()(_add_x_and_y_and_z)

# workgraph = write_workflow_json(filename='workflow_simple.json')

# TODO: Create inputs rather than tasks out of data nodes
wg = WorkGraph('wg-simple')

helper_task1 = wg.add_task(
    helper_1,
    name="x",
    value=1
)

helper_task2 = wg.add_task(
    helper_2,
    name="y",
    value=2
)

add_x_and_y_task = wg.add_task(
    add_x_and_y,
    name='add_x_and_y',
    x=helper_task1.outputs.result,
    y=helper_task2.outputs.result,
)

add_x_and_y_and_z_task = wg.add_task(
    add_x_and_y_and_z,
    name='add_x_and_y_and_z',
    x=add_x_and_y_task.outputs.x,
    y=add_x_and_y_task.outputs.y,
    z=add_x_and_y_task.outputs.z,
)

wg

outputs [{'identifier': 'workgraph.any', 'name': 'result'}]


NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'wg-simp…

In [10]:
write_workflow_json(wg=wg, file_name=workflow_json_filename)


{'nodes': {0: 1,
  1: 2,
  2: 'simple_workflow.add_x_and_y',
  3: 'simple_workflow.add_x_and_y_and_z'},
 'edges': [{'source': 0,
   'target': 2,
   'sourceHandle': None,
   'targetHandle': 'x'},
  {'source': 1, 'target': 2, 'sourceHandle': None, 'targetHandle': 'y'},
  {'source': 2, 'target': 3, 'sourceHandle': 'x', 'targetHandle': 'x'},
  {'source': 2, 'target': 3, 'sourceHandle': 'y', 'targetHandle': 'y'},
  {'source': 2, 'target': 3, 'sourceHandle': 'z', 'targetHandle': 'z'}]}

In [11]:

flow = load_workflow_json(file_name=workflow_json_filename)

In [12]:
result = run_locally(flow)
result

2025-03-15 11:14:35,722 INFO Started executing jobs locally
2025-03-15 11:14:35,874 INFO Starting job - add_x_and_y (0ea991be-405e-4c9f-8ac7-0c34e7b40d03)
2025-03-15 11:14:35,878 INFO Finished job - add_x_and_y (0ea991be-405e-4c9f-8ac7-0c34e7b40d03)
2025-03-15 11:14:35,879 INFO Starting job - add_x_and_y_and_z (0a4613e0-432b-49fe-b0cc-48a057667532)
2025-03-15 11:14:35,881 INFO Finished job - add_x_and_y_and_z (0a4613e0-432b-49fe-b0cc-48a057667532)
2025-03-15 11:14:35,882 INFO Finished executing jobs locally


{'0ea991be-405e-4c9f-8ac7-0c34e7b40d03': {1: Response(output={'x': 1, 'y': 2, 'z': 3}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/geiger_j/aiida_projects/adis/git-repos/python-workflow-definition'))},
 '0a4613e0-432b-49fe-b0cc-48a057667532': {1: Response(output=6, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/geiger_j/aiida_projects/adis/git-repos/python-workflow-definition'))}}